### Building Jupyter Kernels Using Simple Python Wrappers
#### Safia Abdalla (@captainsafia)

Hi, there! You've probably gotten to this Notebook via the tutorial I presented at PyData Chicago 2016. If not, be sure to check out the video for this talk on YouTube so that you have the right context. If you have, read on and enjoy this Notebook.

In the talk, we covered the basic structure of the Jupyter Messaging Protocol. In this notebook, we are going to take the ideas that we covered and apply them in the development of a simple Jupyter Kernel for Bash command line. This is adapted from the work done by Jupyter developer, Thomas Kluyver in the [bash_kernel](https://github.com/takluyver/bash_kernel).

To build this kernel, we will be subclassing the IPython `Kernel` class and overriding some of its default functions. Let's start off by importing the class from the IPython library.

In [1]:
from ipykernel.kernelbase import Kernel

Next we'll create a `BashKernel` class that subclasses the `Kernel` that we just imported. We'll set two default attributes on this class, the implementation name, implementation version, the langauge, and the language version.

We'll use these variables to create a `banner` variable which stores the text that is displayed to the user on kernel start if they are using a console UI.

In [2]:
class MyBashKernel(Kernel):
  implementation = ''
  implementation_version = ''
  language = ''
  language_version = ''
  banner = language + language_version + '\n'

The next attribute that we will need to add is the `langauge_info` attribute. This stores information about the langauge in the form of a dictionary. If you've ever poked around the raw JSON of a notebook, you'll notice a `language_info` key which contains some information about the language the kernel that your notebook uses is associated with. We'll set this dictionary in our `MyBashKernel` class.

In [3]:
class MyBashKernel(Kernel):
  implementation = ''
  implementation_version = ''
  language = ''
  language_version = ''
  banner = language + language_version + '\n'
  language_info = {
    'name': 'bash',
    'codemirror_mode': 'shell',
    'mimetype': 'text/x-sh',
    'file_extension': '.sh'
  }

You'll not that the language_info dictionary provides information to CodeMirror, the code editor utilized by the Jupyter front-end about what syntax to use for code cells in this notebook. The language_info dictionary also specifies the file extension that should be used for files exported from this notebook.

Let's start off by creating the `__init__` function for our class, this function will instantiate the `Kernel` parent class and initialize the connection to the Bash REPL using `pexpect`.

In [4]:
def __init__(self, **kwargs):
    Kernel.__init__(**kwargs)
    self._start_bash()
    
def _start_bash(self):
    from pexpect import replwrap, EOF
    import signal
    
    sig = signal.signal(signal.SIGINT, signal.SIG_DFL)
    try:
        self.bash_wrapper = replwrap.bash()
    finally:
        signal.signal(signal.SIGINT, sig)

Now that we've got our connection to the Bash REPL instantiated, we can move on to writing some of the logic for code execution. In order to do this, we will need to override the `do_execute` function, which as the name might suggest, handles what the kernel will do when sent a request to execute some code.

This parameters passed to the function include the `code` that needs to executed, whether or not to display the output from the code exeuction (`silent`), whether to increment the execution counter after the code has been executed (`store_history`), what instructions from the user to run afterwards (`user_expressions`), and whether to accept input from the user (`allow_stdin`).

The function returns a dictionary which might contain some of the following keys.

```
{
  'status' : str,
  'execution_count' : int,
  'payload' : list(dict),
  'user_expressions' : dict,
  'ename' : str,
  'evalue' : str,
  'traceback' : list,
}
```

`status` can either be "OK" or "error" or "abort". If `status` is "OK", the `execution_count` is incremented by 1. `payload` is a deprecated feature so for now, you can return an empty list. Finally, `user_expressions` should contain the result of executing any of the expressions passed in via the `user_expressions` parameter to the `do_execute` function.

If the `status` is "ERROR", the error name (`ename`), error value (`evalue`), and error traceback (`traceback`) must be provided.

We'll start off by checking to see if we are provided a `code` to execute. If not, we will return an empty payload with an "OK" status.

In [5]:
def do_execute(self, code, silent, store_history=True, user_expressions=None, allow_stdin=False):
  if not code.strip():
    return {
      'status': 'OK',
      'execution_count': self.execution_count,
      'payload': [],
      'user_expressions': {}
    }

Our reference to `self.execution_count` is valid in this case although we did not define such an attribute in our class definition because the call to the `__init__` on the parent `Kernel` class took care of initializing this variable.

Now, if the user did provide us `code` to run, we will need to execute it using our connection to the Bash REPL made using `bash_wrapper`, so we'll add the following code to our `do_execute` function.

In [6]:
def do_execute(self, code, silent, store_history=True, user_expressions=None, allow_stdin=False):
    interrupted = False
    try:
        output = self.bashwrapper.run_command(code.rstrip(), timeout=None)
    except KeyboardInterrupt:
        self.bash_wrapper.child.sendintr()
        interrupted = ____
        self.bash_wrapper._expect_prompt()
        output = self.bash_wrapper.child.before
    except EOF:
        output = self.bash_wrapper.child.before + 'Restarting Bash'
        self._start_bash()


In this code, we send our `code` for execution by the Bash REPL. If the user interrupts the command via the keyboard we send an interrupt signal to the REPL and reset the prompt. Alternatively, if the user sends an EOF signal by pressing Ctrl + D on the keyboard, we restart the Bash REPL. Now that we have executed the command, we will need to return the `output` back to the user. We'll only need to do this if the user has set `silent` to `False`. Let's go ahead add the logic for sending back the output from the kernel to the front-end.

In [7]:
def do_execute(self, code, silent, store_history=True, user_expressions=None, allow_stdin=False):
    if not _____:
        stream_content = {'name': 'stdout', 'text': output}
        self.send_response(self.iopub_socket, 'stream', stream_content)

Believe it or not, these two lines are extremely powerful. We create a packaged `stdout` text response and send it via the iopub socket to the front-end.

If the user interrupted the currently running command, we are going to want to return a different message with an "abort" `status`. We'll use that `interrupted` Boolean that we set earlier.

In [8]:
def do_execute(self, code, silent, store_history=True, user_expressions=None, allow_stdin=False):
    if interrupted:
        return ____

So far, so good! But there is one thing that we forgot to check for. Errors! What if the user enters a bad shell command, we'll need a way to respond to them with this. We can check if any errors occured by requesting the REPL execute a trivial command and seeing if it sends us back a response. If it doesn't, that means its hung up on an error from the previous command.

The trivial command we will have the REPL execute is `echo $?`, which outputs 0 to the shell. If it doesn't work we will need to return an error response to the front-end.

In [9]:
def do_execute(self, code, silent, store_history=True, user_expressions=None, allow_stdin=False):
    try:
        exitcode = int(self.bash_wrapper.run_command('echo $?').rstrip())
    except Exception:
        exitcode = 1

    if exitcode:
        error_content = {'execution_count': self.execution_count,
                         'ename': '', 'evalue': str(exitcode), 'traceback': []}

        self.send_response(self.iopub_socket, 'error', error_content)
        error_content['status'] = '_______'
        return error_content
    else:
        return {'status': 'ok', 'execution_count': self.execution_count,
                'payload': [], 'user_expressions': {}}

If we put all this together, our `do_execute` function will look like this.

In [21]:
def do_execute(self, code, silent, store_history=True, user_expressions=None, allow_stdin=False):
    if not code.strip():
        return {
          'status': 'OK',
          'execution_count': self.execution_count,
          'payload': [],
          'user_expressions': {}
        }
    
    interrupted = False
    try:
        output = self.bashwrapper.run_command(code.rstrip(), timeout=None)
    except KeyboardInterrupt:
        self.bash_wrapper.child.sendintr()
        interrupted = True
        self.bash_wrapper._expect_prompt()
        output = self.bash_wrapper.child.before
    except EOF:
        output = self.bash_wrapper.child.before + 'Restarting Bash'
        self._start_bash()
    
    if not silent:
        stream_content = {'name': 'stdout', 'text': output}
        self.send_response(self.iopub_socket, 'stream', stream_content)
        
    if interrupted:
        return _____
    
    try:
        exitcode = int(self.bash_wrapper.run_command('echo $?').rstrip())
    except Exception:
        exitcode = 1

    if exitcode:
        error_content = {'execution_count': self.execution_count,
                         'ename': '', 'evalue': str(exitcode), 'traceback': []}

        self.send_response(self.iopub_socket, 'error', error_content)
        error_content['status'] = 'error'
        return error_content
    else:
        return {'status': 'ok', 'execution_count': self.execution_count,
                'payload': [], 'user_expressions': {}}

And that's it for our `do_execute` function. Our entire `MyBashKernel` code will look like this.

In [22]:
class MyBashKernel(Kernel):
    implementation = ''
    implementation_version = ''
    language = ''
    language_version = ''
    banner = language + language_version + '\n'
    language_info = {
        'name': 'bash',
        'codemirror_mode': 'shell',
        'mimetype': 'text/x-sh',
        'file_extension': '.sh'
    }
    
    def __init__(self, **kwargs):
        Kernel.__init__(**kwargs)
        self._start_bash()

    def _start_bash(self):
        from pexpect import replwrap, EOF
        import signal

        sig = signal.signal(signal.SIGINT, signal.SIG_DFL)
        try:
            self.bash_wrapper = replwrap.bash()
        finally:
            signal.signal(signal.SIGINT, sig)

    def do_execute(self, code, silent, store_history=True, user_expressions=None, allow_stdin=False):
        if not code.strip():
            return {
              'status': 'OK',
              'execution_count': self.execution_count,
              'payload': [],
              'user_expressions': {}
            }

        interrupted = False
        try:
            output = self.bashwrapper.run_command(code.rstrip(), timeout=None)
        except KeyboardInterrupt:
            self.bash_wrapper.child.sendintr()
            interrupted = True
            self.bash_wrapper._expect_prompt()
            output = self.bash_wrapper.child.before
        except EOF:
            output = self.bash_wrapper.child.before + 'Restarting Bash'
            self._start_bash()

        if not ____:
            stream_content = {'name': 'stdout', 'text': output}
            self.send_response(self.iopub_socket, 'stream', stream_content)

        if interrupted:
            return _____

        try:
            exitcode = int(self.bash_wrapper.run_command('echo $?').rstrip())
        except Exception:
            exitcode = 1

        if exitcode:
            error_content = {'execution_count': self.execution_count,
                             'ename': '', 'evalue': str(exitcode), 'traceback': []}

            self.send_response(self.iopub_socket, 'error', error_content)
            error_content['status'] = 'error'
            return error_content
        else:
            return {'status': 'ok', 'execution_count': self.execution_count,
                    'payload': [], 'user_expressions': {}}

There are several other functions that we can override from our `Kernel` parent class, including `do_complete` which handles how tab completion is done and `do_history` which handles how history is accessed. These are optional and will add more functionality to our application. For now, we've got the minimum code required that we need to get this working. The next thing that we will need is configure the installation information for our kernel. We can do this in a seperate file. This file outlines the kernelspec which contains the name of the kernel, the version, and how the kernel is run.

Remember in addition to developing kernels in Python you can also develop them in the native kernel langauge by leveraging the ZMQ binding library in that langauge.

#### Thanks for reading this notebook!